In [12]:
from typing import  Iterator,  MutableSequence

def tokenize(stem: str) -> tuple[MutableSequence[str | None], MutableSequence[str]]:
    tokens = stem.split("_")
    keys: MutableSequence[str | None] = list()
    values: MutableSequence[str] = list()
    for token in tokens:
        if "-" in token:  # A bids tag
            key: str | None = token.split("-")[0]
            if key is None:
                continue
            keys.append(key)
            values.append(token[len(key) + 1 :])

        else:  # A suffix
            keys.append(None)
            values.append(token)
    return keys, values

def parse(phenotype: str) -> Iterator[tuple[str, str]]:
    keys, values = tokenize(phenotype)

    for key, value in zip(keys, values, strict=False):
        if key is None:
            continue
        yield (key, value)

In [19]:
from collections import defaultdict
import zipfile
from pathlib import Path

base_path = Path("/scratch/imaging/consistency-check/data")

image_paths_by_sub_and_feature = defaultdict(lambda: defaultdict(list))
for zip_file_path in base_path.glob("*.zip"):
    try:
        zip_path = zipfile.Path(zip_file_path)
    except zipfile.BadZipFile:
        continue
    for image_path in zip_path.glob("**/*SeedCorr*.nii.gz"):
        tags = dict(parse(image_path.stem))
        sub = tags["sub"]
        feature = tags["feature"]
        image_paths_by_sub_and_feature[sub][feature].append(image_path)

In [24]:
from itertools import chain


image_paths_by_sub_and_feature.keys(), set(chain.from_iterable(x.keys() for x in image_paths_by_sub_and_feature.values()))

(dict_keys(['9040', '09', '13192', '01']),
 {'FalseComb0SeedCorr',
  'FalseComb1SeedCorr',
  'FalseComb2SeedCorr',
  'FalseComb3SeedCorr',
  'FalseComb4SeedCorr',
  'FalseComb5SeedCorr',
  'TrueComb0SeedCorr',
  'TrueComb1SeedCorr',
  'TrueComb2SeedCorr',
  'TrueComb3SeedCorr',
  'TrueComb4SeedCorr',
  'TrueComb5SeedCorr'})

In [28]:
sub = "9040"
feature = "FalseComb2SeedCorr"

image_paths = image_paths_by_sub_and_feature[sub][feature]
len(image_paths)

100

In [2]:
import nibabel as nib

images = [nib.nifti1.load(image_path) for image_path in image_paths]

ModuleNotFoundError: No module named 'nibabel'